## **Cyclone Tutorial Notebook**
### *Architecting Highly Parallel Fault-Tolerant QCCD Systems*

Welcome to the **Cyclone tutorial notebook**. This notebook provides an end-to-end walkthrough of the computational workflow used in the paper.

---

### **Overview**

This notebook will walk through the following steps:

1. **Baseline Execution Extraction**  
   We begin by running the baseline pipeline from  
   _“Architecting Noisy Intermediate-Scale Trapped Ion Quantum Computers”_  ([arXiv:2004.04706](https://arxiv.org/pdf/2004.04706)) to collect execution data for both **bivariate bicycle (BB)** codes and  **hypergraph product (HGP)** codes. The results are stored in a dictionary for reuse in later stages.

2. **Cyclone Compilation**  
   Using the **Cyclone compiler**, we convert each code instance into an execution schedule and extract the associated **time arrays** corresponding to each code's execution time with Cyclone for hypergraph product codes compared to the baseline stored dictionary values, and for BB codes we do this part directly.

3. **Logical Error Rate (LER) Simulation**  
   With the time arrays in hand (and some formatting to create the baseline time array from the dictionary in step #1), we demonstrate how to run LER simulations. Note how HGP codes and BB codes have a seperate workflow. Notably, the BB codes in this notebook are created through functions in the BB_compilers.py file, and output LERs to files in different directories requiring **manual** insertion of plotting data, whereas the HGP uses the cyclone_compiler.py file and we store the output as a variable to use for plotting.  *Because LER simulations involve sampling, small variations from the paper’s figures may occur; increasing the number of samples reduces variance.*

---

### **Requirements**

- Python ≥ 3.10  
- Associated Cyclone repo/compiler
- A machine with moderate compute resources

---

### **Notebook Structure**

- **Section 0:** Environment Configuration
- **Section 1:** Baseline data extraction  (30 minutes)
- **Section 2:** Cyclone compilation and LER simulation for HGP codes (~6 hours) 
- **Section 3:** Cyclone compilation and LER simulation for BB codes (~8 hours)



### **Section 0: Install Requirements and Set Up Your Environment**

We can install all the necessary requirements with the following command


In [ ]:
###setup
!pip install -r requirements.txt


### **Section 1: Baseline Execution Time Extraction**  

The baseline is run by creating a grid (more machine types in test_machines.py, and to change these one would have to change machine string logic and run.py). A temporary output json is used to track incremental progress. The final dictionary, baseline_values, will be used for the rest of the notebook to reference baseline execution times, and should match with Figure 19 of the Cyclone manuscript.

The whole cell should run in ~30 minutes - 1 hour.

In [ ]:
###Baseline Setup


import subprocess as sp
import os
import datetime
import math
import json
PROG = []
code_indentifiers = [] #NOTE: relative order must be preserved throughout code. Do not change this logic.
for x in os.listdir("QLDPC_Codes"):
    identifier = int(x.split("[[")[1].split(",")[0]) #the number of data qubits in the benchmark
    if (identifier != 1225): #skip this benchmark for time sake, too large when it gets to performing memory experiments on small p.
        PROG.append("QLDPC_Codes/" + x)
        code_indentifiers.append(identifier)
output_file = open('baseline_output.log','w')



#dictionary format is used to represent the situation where one might want to vary individual capacities. The default case is the simple constant of 5 ions per trap.
ion_constants = {225: 5, 375: 5, 625: 5, 72: 5, 90: 5, 144: 5}

mapper = "Greedy"
reorder = "Naive"
count = 0  
print("PROG", PROG)
baseline_values = {}
for p in PROG:
    n = code_indentifiers[count]
    MACHINE=[f"Gx{math.ceil(math.sqrt(n))}"] #substitute with sqrt(n) x sqrt(n) data qubits grid
    IONS = [str(ion_constants[n])]# capacities from above commented capacities
    for m in MACHINE:
        for i in IONS:
           print("Starting", p, datetime.datetime.now())
           code = sp.call(["python", "run.py", p, m, i, mapper, reorder, "1", "0", "0", "FM", "GateSwap"], stdout=output_file)
           if (code != 0):
               print("There was an error running the baseline. Please debug outside notebook in the run_batch.py (baseline) file")
           with open("tmp_output.json") as f:
               baseline_values[n] = json.load(f)[p]
    print("baseline values here", baseline_values)
           #sp.call(["python", "run_junction_network.py", p, m, i, mapper, reorder, "1", "0", "0", "FM", "GateSwap"])#, stdout=output_file)
    count += 1
           


In [ ]:
### GET QLDPC Code Cyclone Execution Times
import cyclone_compiler
import math 
import numpy as np

codes = []

#""" QLPDC RUNNABLE BLOCK
h1 = np.loadtxt('quits/parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt', dtype=int)
qldpc1 = cyclone_compiler.generate_code_qldpc(h1) 
h2 = np.loadtxt('quits/parity_check_matrices/n=20_dv=3_dc=4_dist=8.txt', dtype=int)
qldpc2 = cyclone_compiler.generate_code_qldpc(h2) 
h = np.loadtxt('quits/parity_check_matrices/n=28_dv=3_dc=4_dist=10.txt', dtype=int)
qldpc3 = cyclone_compiler.HgpCode(h2, h1)         # Define the HgpCode object
qldpc3.build_graph(seed=22)
codes.append(qldpc1) #[[225,9,6]]
codes.append(qldpc2) #[[375,15,6]]
codes.append(qldpc3) #[[625,25,8]]



qldpc_cyclone_timings = []
for code in codes:
        n_data_qubits = len(code.data_qubits) #if QLDPC this line
        #n_data_qubits = code.N ###if CSS this line
        
        space_efficient_mode = True
        sensitivity_mode = True
        num_logical = code.lz.shape[0]
        num_zcheck, num_data = code.hz.shape
        num_xcheck, num_data = code.hx.shape
        assert(n_data_qubits == num_data)
        num_ancilla = num_xcheck + num_zcheck
        name = f"{num_data}-{num_logical}-x"

        CUSTOM_TRAP_AMOUNT = num_ancilla/2
        #CUSTOM_TRAP_AMOUNT = 121
        CUSTOM_ION_AMOUNT = math.ceil(n_data_qubits/CUSTOM_TRAP_AMOUNT) + math.ceil(num_ancilla/CUSTOM_TRAP_AMOUNT)
        COMBINED_SHUTTLE_TIME = 165 #COMBINED_SHUTTLE_TIME = 165 #80*2 (split/merge) + 5 (move across degree 2 junction)
        ionSWAP = False #False means GateSWAP (default)
        
        input_tuple = cyclone_compiler.get_input_tuple_from_code_qldpc(code)
        max_weight = cyclone_compiler.max_stabilizer_weight(input_tuple) #this doesn't matter anymore
        rows = len(input_tuple[2])
        #print("Max stabilizer weight", max_weight)
        print("Rows", rows)
        num_cyclone_ancilla = rows

        machStart = "C"
        mpar_model1 = cyclone_compiler.MachineParams()
        mpar_model1.alpha = 0.003680029
        mpar_model1.beta = 39.996319971
        mpar_model1.split_merge_time = 80
        mpar_model1.shuttle_time = 5
        mpar_model1.junction2_cross_time = 5
        mpar_model1.junction3_cross_time = 100
        mpar_model1.junction4_cross_time = 120
        mpar_model1.gate_type = "FM"
        mpar_model1.swap_type = "GateSwap" #this is just for the machine, don't adjust the swap type here
        mpar_model1.ion_swap_time = 42

        num_ions_per_region = CUSTOM_ION_AMOUNT
        trap_amount = CUSTOM_TRAP_AMOUNT
        m = cyclone_compiler.make_circle_machine_length_n(num_ions_per_region, mpar_model1, int(trap_amount))
        machString = machStart + str(int(trap_amount))

        #NOTE: One can change experimental parameters being passed to the hypergraph product compiler here
        circ, timing, cxs = cyclone_compiler.cyclone_official_compiler(name=name, input_tuple=input_tuple, machine = m, machString=machString, numAncilla=num_cyclone_ancilla, ions=num_ions_per_region, shuttling_times=COMBINED_SHUTTLE_TIME, ionSWAP=ionSWAP)
        qldpc_cyclone_timings.append(sum(timing))
print("-------------------------------")
print("FINAL TIMINGS QLDPC: ")
print(qldpc_cyclone_timings)


### **Section 2:** Cyclone compilation and LER simulation for HGP codes

Cyclone compilation (under a minute)
Here we use the quits library to first make the code objects for the HGP code, then the cyclone compiler is imported to the notebook, and uses the parity check matrices from the HGP code to schedule. Cyclone outputs a list of times where each individual entry is a micro-op/stage of cyclone (gate, split, merge, SWAP), and the sum is the total execution time. We use this array of execution times, qldpc_cyclone_timings, to pass to the next part. 

LER Simulation (~4 hours at the given precision level):
The time intensive part is the LER simulation. This process is *non-deterministic*, so there may be slight variations to the plots produced in the paper, but if the number of shots is sufficiently high, this is not a problem. Noting this tradeoff between time to completion and accuracy, one must choose the number of shots wisely for custom physical error rates(please ensure that significantly increasing the amount of shots multiple times results in the same value). Feel free to play around with the number of shots as time permits and ensure the plot converges to the stable solution (as seen in Figure 15 of the paper) 

In [ ]:
#Plot QLDPC
from quits.qldpc_code import *
from quits.circuit import get_qldpc_mem_circuit
from quits.decoder import sliding_window_bposd_circuit_mem
from quits.simulation import get_stim_mem_result
import stim



def calculate_pauli_twirling(t, p):
    if (p != 0): #log fit to real hardware device t1/t2 times as described in Evaluation Section
        logT1 = -0.99998246709278*math.log(p) + 9.2104211120632
        logT2 = -0.99998246709278*math.log(p) + 9.2104211120632
        T1 = math.exp(logT1)
        T2 = math.exp(logT2)
        x = (1 - math.exp(-t/T1))/4
        y = x
        z = (1-math.exp(-t/T2))/2 - (1 - math.exp(-t/T1))/4
        return x, y, z
    else:
        return 0, 0, 0

def run_LER(circuit):
    num_trials = 100
    ##This tree of different shot numbers was made with optimizing time in mind. Feel free to strictly increase the number of shots anywhere across the board.
    if (cyclone_mode and code_index == 0 or cyclone_mode and code_index == 2):
        num_trials = 5000
    elif (cyclone_mode or code_index == 0):
        num_trials = 500
    else:
        num_trials = 100


    detection_events, observable_flips = get_stim_mem_result(circuit, num_trials, seed=1)   # simulate the circuit using Stim

    W, F = 5, 3                     # sliding window parameters
    max_iter, osd_order = 20, 10    # BP-OSD decoder parameters 

    logical_pred = sliding_window_bposd_circuit_mem(detection_events, circuit, code.hz, code.lz,\
                                                    W, F, max_iter=max_iter, osd_order=osd_order, tqdm_on=True)

    pL = np.sum((observable_flips- logical_pred).any(axis=1)) / num_trials
    lfr = 1 - (1-pL)**(1/num_rounds)
    print('p: %.7f, LFR: %.7f'%(p, lfr))
    return lfr


times = qldpc_cyclone_timings #every index maps to the code (so this is really a primitive way of doing a hashmap where code --> time)

error_rates = [6e-5, 8e-5, 1e-4]

cyclone_mode = True
cyclone_code_LERs_qldpc = []
assert(len(codes) == len(times))
for code_index in range(len(codes)):
    code = codes[code_index]
    LERS = []
    print("On code", code_index)
    for error_index in range(len(error_rates)):
        p = error_rates[error_index]
        time = times[code_index]
        injected_p, _, _ = calculate_pauli_twirling(time, p) #X and Y and Z
        print("Injected p", injected_p)
        p = injected_p + p
        print("Total physical error rate", p)
        num_rounds = 15    # number of rounds (T-1)
        basis = 'Z'        # 'Z' or 'X'

        circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))

        # print(circuit)
        lfr = run_LER(circuit)
        print("Got LER", lfr)
        LERS.append(lfr)
    print("X axis", error_rates)
    print("LER for code", code_index)
    print(LERS)
    cyclone_code_LERs_qldpc.append(LERS)

print("TOTAL LERS Cyclone", cyclone_code_LERs_qldpc)

cyclone_mode = False
baseline_code_LERs_qldpc = []
baseline_times = []
baseline_times.append(baseline_values[225])
baseline_times.append(baseline_values[375])
baseline_times.append(baseline_values[625])
assert(len(codes) == len(baseline_times))
for code_index in range(len(codes)):
    code = codes[code_index]
    LERS = []
    print("On code", code_index)
    for error_index in range(len(error_rates)):
        p = error_rates[error_index]
        time = baseline_times[code_index]
        injected_p, _, _ = calculate_pauli_twirling(time, p) #X and Y and Z
        print("Injected p", injected_p)
        p = injected_p + p
        print("Total physical error rate", p)
        num_rounds = 15    # number of rounds (T-1)
        basis = 'Z'        # 'Z' or 'X'

        circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))

        # print(circuit)
        lfr = run_LER(circuit)
        print("Got LER", lfr)
        LERS.append(lfr)
    print("X axis", error_rates)
    print("LER for code", code_index)
    print(LERS)
    baseline_code_LERs_qldpc.append(LERS)

print("TOTAL LERS Baseline", baseline_code_LERs_qldpc)




    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_cyclone_vs_baseline_ler():
    code_params = [
        [225, 9, 6],
        [375, 15, 6],
        [625, 25, 8]
    ]
    
    cyclone_lers = cyclone_code_LERs_qldpc
    

    baseline_lers = baseline_code_LERs_qldpc
    
    plt.figure(figsize=(10, 6))

    markers = ['o', 's', '^']
    num_colors = 5

    viridis = cm.get_cmap('viridis', num_colors)
    colors = viridis(np.linspace(0, 1, num_colors))
    colors = [colors[0], colors[1], colors[3]]

    for i, (params, cyclone, baseline) in enumerate(zip(code_params, cyclone_lers, baseline_lers)):
        label_base = f"B[[{params[0]}, {params[1]}, {params[2]}]]"
        label_cyclone = f"C[[{params[0]}, {params[1]}, {params[2]}]]"
        x = error_rates
        
        plt.plot(x, baseline, linestyle='--', color=colors[i], marker=markers[i], fillstyle='none', label=label_base)
        plt.plot(x, cyclone, linestyle='-', color=colors[i], marker=markers[i], label=label_cyclone)

    plt.yscale('log')
    plt.xlabel('Physical Error Rate')
    plt.ylabel('Logical Error Rate (LER)')
    plt.xticks(error_rates, ["$6\\times10^{-5}$", "$8\\times10^{-5}$", "$1\\times10^{-4}$"],rotation=20)
    plt.yticks([1e-0, 1e-1, 1e-2,1e-3,1e-4,1e-5,1e-6], ["$10^{-0}$", "$10^{-1}$", "$10^{-2}$", "$10^{-3}$", "$10^{-4}$", "$10^{-5}$", "$10^{-6}$"])
    plt.legend(fontsize=6, frameon=False)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.gcf().set_size_inches(5.0, 2.5)
    plt.tight_layout()
    plt.show()

plot_cyclone_vs_baseline_ler()

### **Section 3:** Cyclone compilation and LER simulation for BB codes

For BB codes, the workflow runs the logical error experiments immediately upon generating the Cyclone/baseline timings and the results are written to CODE_X/ directory where X is the code being executed follow by the results file of baseline (results_baseline) or Cyclone (results). 

For example, CODE_144_12_12/result file will contain the results in a table format for the baseline at the [[90,8,10]] bivariate bicycle code like so:

base_error     hardware_error_rate num_cycles     n(E)    logical_error_rate
0.00125	0.0037951738429019546	12	500	17	0.034
0.0015	0.004551081040863129	12	500	39	0.078
0.00175	0.005305952459873279	12	500	128	0.256

Upon the completion of the below cell, all Cyclone and baseline results will be written to their respective files. One can then take this data and **manually** fill in the following plotting script cell. The x axis should mirror the values of error_rates (also base error in the table output), and the corresponding y axis depending on the code prefix and whether or not it is cyclone or baseline. Continuing on the above example, the manual insertion to the plotting script would look like: 

p_data = [1.25e-3, 1.5e-3, 1.75e-3] \
y_144data = [0.0037951738429019546, 0.004551081040863129, 0.005305952459873279]

Once the data is gathered, the plotting script uses the few sampled values to extrapolate the overall LER, to the fit as detailed in https://www.nature.com/articles/s41586-024-07107-7

In [ ]:
import BB_compilers




#input physical error rates (x axis)
error_rates = [1.25e-3, 1.5e-3, 1.75e-3]
#adjust num_shots as time allows/precision desired
num_shots = 1000

#Cyclone runs, traps = m/2

BB_compilers.run_cyclone_BB_Compiler("72-12-6", 36, error_rates=error_rates, num_shots=num_shots) 
BB_compilers.run_cyclone_BB_Compiler("90-8-10", 45, error_rates=error_rates, num_shots=num_shots) 
BB_compilers.run_cyclone_BB_Compiler("144-12-12", 72, error_rates=error_rates, num_shots=num_shots) 

baseline_values = {144: 619710, 225: 502675, 375: 806580, 625: 1331795, 72: 292540, 90: 187170}
bb_code_timings_baseline = [baseline_values[72], baseline_values[90], baseline_values[144]]
BB_compilers.run_baseline_numbers(baseline_timings=bb_code_timings_baseline, error_rates=error_rates, num_shots=num_shots)


In [ ]:
#plot Cyclone values
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def log_model(p, c0, c1, c2):
    return 3*np.log(p) + c0 + c1 * p + c2 * p**2

def fit_curve(p, y):
    p = np.asarray(p)
    y = np.asarray(y)
    log_y = np.log(y)
    popt, pcov = curve_fit(log_model, p, log_y, p0=(0.0, 0.0, 0.0))
    return popt, pcov

def model(p, c0, c1, c2):
    return np.exp(np.log(p**3) + c0 + c1 * p + c2 * p**2)

def plot_BB_code_LERS():
    
    #TODO: Insert Data Here as produced in the corresponding CODE_X/result files. 
    # Cyclone results are written to result, baseline are written to result_baseline
    #Detailed instructions and example given in Section 3 header description
    
    p_data = [1.25e-3, 1.5e-3, 1.75e-3]
    y_72data = [0.0076, 0.0108, 0.0196]
    y_72_baseline = [0.321, 	0.548, 0.727]
    y_90data = [0.0013, 0.005, 0.007]
    y_90_baseline = [0.633, 0.895, 0.987]
    y_144data = [0.0015, 0.009, 0.023]
    y_144_baseline = [0.48, 0.82, 1]
    

    
    ###End of data changing part
  
        
    params72, covariance72 = fit_curve(p_data, y_72data)
    params90, covariance90 = fit_curve(p_data, y_90data)
    params144, covariance144 = fit_curve(p_data, y_144data)
    params72baseline, covariance72b = fit_curve(p_data, y_72_baseline)
    params90baseline, covariance90b = fit_curve(p_data, y_90_baseline)
    params144baseline, covariance144b = fit_curve(p_data, y_144_baseline)

    #bparams, bcovariance = fit_curve(p_baseline, y_baseline)
    #scparams, sccovariance = fit_curve(p_sc, y_sc)

    # Predict values for smooth curve
    #p_fine = np.linspace(1e-6, max(p_data), 500)
    p_fine = np.linspace(min(p_data)*0.2, max(p_data), 500)
    y_fit72 = model(p_fine, *params72)
    y_fit90 = model(p_fine, *params90)
    y_fit144 = model(p_fine, *params144)
    y_fit72b = model(p_fine, *params72baseline)
    y_fit90b = model(p_fine, *params90baseline)
    y_fit144b = model(p_fine, *params144baseline)
    
    num_colors = 5

    # Get the Viridis colormap
    viridis = cm.get_cmap('viridis', num_colors)

    # Generate the 5 colors.
    # viridis(np.linspace(0, 1, num_colors)) gives us the RGBA values directly.
    colors = viridis(np.linspace(0, 1, num_colors))
    colors = [colors[0], colors[1], colors[3]]
    # Plotting
    plt.figure(figsize=(8, 5))
    plt.scatter(p_data, y_72data, color=colors[0])
    plt.plot(p_fine, y_fit72, label='C[[72,12,6]]', color=colors[0], linewidth=2)
    
    plt.scatter(p_data, y_90data, color=colors[1])
    plt.plot(p_fine, y_fit90, label='C[[90,8,10]]', color=colors[1], linewidth=2)
    plt.scatter(p_data, y_144data, color=colors[2])
    plt.plot(p_fine, y_fit144, label='C[[144,12,12]]', color=colors[2], linewidth=2)
    plt.scatter(p_data, y_72_baseline, color=colors[0])
    plt.plot(p_fine, y_fit72b, label='B[[72,12,6]]', color=colors[0], linewidth=2, linestyle="--")
    plt.scatter(p_data, y_90_baseline, color=colors[1])
    plt.plot(p_fine, y_fit90b, label='B[[90,8,10]]', color=colors[1], linewidth=2, linestyle="--")
    plt.scatter(p_data, y_144_baseline, color=colors[2])
    plt.plot(p_fine, y_fit144b, label='B[[144,12,12]]', color=colors[2], linewidth=2, linestyle="--")
    plt.yscale('log')
    plt.xscale('log')
    plt.xticks([2.5e-4, 5e-4, 7.5e-4, 1e-3, 2e-3], ["$2.5\\times10^{-4}$", "$5\\times10^{-4}$", "$7.5\\times10^{-4}$", "$10^{-3}$", "$2 \\times 10^{-3}$"], rotation=45)
    plt.yticks([1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7], ["$10^{-1}$", "$10^{-2}$", "$10^{-3}$", "$10^{-4}$", "$10^{-5}$", "$10^{-6}$", "$10^{-7}$"])
    plt.xlabel('Physical Error Rate')
    plt.ylabel('y')

    #plt.title('BB Code Comparison')
    plt.grid(True)
    plt.tight_layout()
    plt.gcf().set_size_inches(5.0, 2.5)
    lgd = plt.legend(fontsize=9, frameon=False, ncols=1, loc="lower right")#, bbox_to_anchor = (1.25, 0.6))
    plt.show()

plot_BB_code_LERS()